# SynthMorph PyTorch Demo

## Purpose
Reproduce the original SynthMorph demo in Torch.
- Data generation
- Registration model training
- Registration (inference) examples  

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import urllib.request
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np 
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import pytorch_lightning as pl

# local code
from synthmorph import models, datamodule as dm, utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'     # note: only gpu has been tested so far
torch.multiprocessing.set_start_method('spawn')

## Synthetic Images
Currrently still using NumPy code to generate synthetic labels and images, Torch implementation is still WIP.

### Generate Label (i.e. Segmentation) Map
Generating labels in NumPy is slow

In [ ]:
# Input shapes.
in_shape = (256,) * 2
num_dim = len(in_shape)
num_label = 4
label_map = dm.generate_map(in_shape, num_label)
plt.imshow(label_map, cmap='tab20c')

### Generate Image from Label Map

In [ ]:
n = 4
gen_images = [dm.map_to_image(label_map) for _ in range (n)]

# Set up the subplot layout
fig, axes = plt.subplots(1, n, figsize=(n*3, 3))

# Iterate over the images and plot them in the subplots
for i in range(n):
    axes[i].imshow(gen_images[i][1], cmap='gray')
    axes[i].axis('off')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.05)

# Display the plots
plt.show()


In [ ]:
# Warning: Labels with larger num_labels take longer to generate
# WIP to use Torch instead of NumPy for label map  generation
size= 40
in_shape = (256,) * 2
num_labels = 4
train_data = dm.SynthMorphDataset(
    size=size,
    input_size=in_shape,
    num_labels=num_labels,
)

dataloader = DataLoader(
    dataset=train_data,
    batch_size=1,
    shuffle=True,
    num_workers=8,
    persistent_workers=True
)

In [ ]:
# Example plot for an image and its labels
results = next(iter(dataloader))
image = results['fixed']
label_map = results['fixed_map']
# Plot each label of an image
plot_num = num_labels + 1
fig, axes = plt.subplots(1, plot_num, figsize=(plot_num*8, 8))
axes[0].imshow(image.squeeze().tolist(), cmap='gray')
axes[0].axis('off')
for c in range(1, plot_num):
    ax = axes[c]
    label = label_map.squeeze().tolist()[c - 1]
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1)
    ax.imshow(label, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])

plt.subplots_adjust(wspace=0.05)
plt.show()

## Model Training

In [ ]:
# You can generate the Torch version of the original author's weights from tf2torch.ipynb
# State_dict weights for the registration model, different from PL checkpoint
reg_weights = None #Path(".") / "torch_weights.pth"   # 'None' for no weight loading
# Fresh model
in_shape = (256,) * 2
unet_enc_nf = [256] * 4
unet_dec_nf = [256] * 8
model = models.SynthMorph(
    vol_size=in_shape,
    num_labels=train_data.num_labels,
    enc_nf=unet_enc_nf,
    dec_nf=unet_dec_nf,
    lmd=1,
    reg_weights=reg_weights
)
n_param = utils.torch_model_parameters(model.reg_model)

In [ ]:
# # Model from checkpoint
# checkpoint_path = './lightning_logs/version_67/checkpoints/epoch=9999-step=400000.ckpt'
# in_shape = (256,) * 2
# unet_enc_nf = [256] * 4
# unet_dec_nf = [256] * 8
# model = model.load_from_checkpoint(
#     checkpoint_path, 
#     vol_size=in_shape,
#     num_labels=train_data.num_labels,
#     enc_nf=unet_enc_nf,
#     dec_nf=unet_dec_nf,
#     lmd=1, 
# )

In [ ]:
max_epochs = 100
steps = train_data.size
max_steps = max_epochs * steps
trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=max_epochs,
    max_steps=max_steps,
    log_every_n_steps=steps
)


In [ ]:
trainer.fit(model=model, train_dataloaders=dataloader)

## Model  Evaluation

In [ ]:
# # Model from best checkpoint
# checkpoint_path = './lightning_logs/version_64/checkpoints/epoch=4999-step=200000.ckpt'
# in_shape = (256,) * 2
# unet_enc_nf = [256] * 4
# unet_dec_nf = [256] * 8
# model = model.load_from_checkpoint(
#     checkpoint_path, 
#     vol_size=in_shape,
#     num_labels=16,
#     enc_nf=unet_enc_nf,
#     dec_nf=unet_dec_nf,
#     lmd=1, 
# )

In [ ]:
model = model.cuda()   # temporary fix
model = model.eval()

### MNIST

In [ ]:
mnist = MNIST(root= "./data", train=False, download=True)

In [ ]:
images = np.array(mnist.data)
labels = np.array(mnist.targets)
indices = np.arange(len(labels))
# Dictionary of indices based on target labels
label_indices_dict = {}
unique_labels = np.unique(labels)
for label in unique_labels:
    label_indices_dict[label] = np.where(labels == label)[0].tolist()

In [ ]:
# Example prediction
in_shape = (256,) * 2
digit = 4
indices = label_indices_dict[digit]
ori_moving = images[indices[6]]
ori_fixed = images[indices[9]]
moving = dm.conform(x=ori_moving, in_shape=in_shape, device=device)
fixed = dm.conform(x=ori_fixed, in_shape=in_shape, device=device)
moved, warp = model.predict_step(moving, fixed)
moved, warp = dm.post_predict(moved), dm.post_predict(warp)
moving, fixed = moving.detach().cpu().numpy(), fixed.detach().cpu().numpy()
moving, fixed = np.squeeze(moving), np.squeeze(fixed)

In [ ]:
movement_plot = [moving, fixed, moved]
movement_headers = ['Moving', 'Fixed', 'Moved']
utils.plot_array_row(movement_plot, movement_headers, cmap='gray')

In [ ]:
warp_plot = [warp[0, ...], warp[1, ...]]
warp_headers=['Warp X-Axis', 'Warp Y-Axis']
utils.plot_array_row(warp_plot, warp_headers, cmap='gray')

### Oasis-1 2D (Brain dataset)

In [ ]:
oasis_path = Path.home() / "oasis_2d"
oasis_path.mkdir(exist_ok=True)
filename = oasis_path / '2D-OASIS-TUTORIAL.npz'
if not filename.exists():
    url = 'https://surfer.nmr.mgh.harvard.edu/pub/data/voxelmorph/2D-OASIS-TUTORIAL.npz'
    urllib.request.urlretrieve(url, filename)
oasis_data = np.load(filename)['images']


In [ ]:
in_shape = (256,) * 2
ori_moving = oasis_data[2]
ori_fixed = oasis_data[7]
moving = dm.conform(x=ori_moving, in_shape=in_shape, device=device)
fixed = dm.conform(x=ori_fixed, in_shape=in_shape, device=device)
moved, warp = model.predict_step(moving, fixed)
moved, warp = dm.post_predict(moved), dm.post_predict(warp)
moving, fixed = moving.detach().cpu().numpy(), fixed.detach().cpu().numpy()
moving, fixed = np.squeeze(moving), np.squeeze(fixed)

In [ ]:
movement_plot = [moving, fixed, moved]
movement_headers = ['Moving', 'Fixed', 'Moved']
utils.plot_array_row(movement_plot, movement_headers, cmap='gray')

In [ ]:
warp_plot = [warp[0, ...], warp[1, ...]]
warp_headers=['Warp X-Axis', 'Warp Y-Axis']
utils.plot_array_row(warp_plot, warp_headers, cmap='gray')

### Affine registration test
Evaluates registration for the following properties of target shapes:
- Size (scaling)
- Coordinates (translation, rotation)

#### Superimposed circles on synthetic image
Note: The label map is only used to create synthetic images which serve as background, only the circles are considered labels (i.e. target shapes)

In [ ]:
# Label generation
in_shape = (256,) * 2
num_dim = len(in_shape)
num_label = 16
label_map = dm.generate_map(in_shape, num_label)

In [ ]:
# Image generation
ori_image = np.squeeze(dm.map_to_image(label_map)[1])
image = ori_image.copy()


In [ ]:
# Example of superimposing circles on image
pixel_value = 255
size_range = (0.030, 0.030)
dist_range = (70, 71)
rotate = 0
x_shift = 0
y_shift = 0
superimposed = utils.superimpose_circles(
    image, 
    pixel_value, 
    size_range, 
    dist_range, 
    rotate,
    x_shift,
    y_shift,
)

superimposed_array = [ori_image, superimposed]
superimposed_headers = ['Original', 'Superimposed']
utils.plot_array_row(superimposed_array, superimposed_headers, cmap='gray')

In [ ]:
# Create random image-mask pairs for moving and fixed
moving_superimpose = lambda moving: utils.superimpose_circles(
    moving, 
    pixel_value=255, 
    size_range=(0.030, 0.030), 
    dist_range=(70, 71), 
    rotate=0,
    x_shift=0,
    y_shift=0,
)
moving= moving_superimpose(np.squeeze(dm.map_to_image(label_map)[1]))

fixed_superimpose = lambda fixed: utils.superimpose_circles(
    fixed, 
    pixel_value=255, 
    size_range=(0.025, 0.035), 
    dist_range=(65, 75), 
    rotate=0,
    x_shift=3,
    y_shift=3,
)
zeros = np.zeros(shape=(256, 256), dtype=np.float32)
fixed_mask = fixed_superimpose(zeros)
fixed = fixed_superimpose(np.squeeze(dm.map_to_image(label_map)[1]))



moving = dm.conform(x=moving, in_shape=in_shape, device=device)
fixed = dm.conform(x=fixed, in_shape=in_shape, device=device)

moved, warp = model.predict_step(moving, fixed)
moved, warp = dm.post_predict(moved), dm.post_predict(warp)
# post-process for plotting
moving, fixed = moving.detach().cpu().numpy(), fixed.detach().cpu().numpy()
moving, fixed = np.squeeze(moving), np.squeeze(fixed)  

In [ ]:
movement_plot = [moving, fixed, moved]
movement_headers = ['Moving', 'Fixed', 'Moved']
utils.plot_array_row(movement_plot, movement_headers, cmap='gray')

In [ ]:
# The labels here are supposed to be the circles only,
#  ignore areas which share the same value
rgb_fixed = utils.convert_to_single_rgb(fixed_mask, 'red')
rgb_moving = utils.convert_to_single_rgb(moving, 'green')
rgb_moved = utils.convert_to_single_rgb(moved, 'blue')

overlay_before = utils.overlay_images(rgb_fixed, rgb_moving)
overlay_after = utils.overlay_images(rgb_fixed, rgb_moved)

overlay_plot = [overlay_before, overlay_after,]
overlay_headers = ['Fixed and Moving', 'Fixed and Moved']
utils.plot_array_row(overlay_plot, overlay_headers, cmap=None)

In [ ]:
warp_plot = [warp[0, ...], warp[1, ...]]
warp_headers=['Warp X-Axis', 'Warp Y-Axis']
utils.plot_array_row(warp_plot, warp_headers, cmap='gray')